# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [12]:
# Write your code below.
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [13]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [14]:
import os
from glob import glob

# Write your code below.
price_data_path = os.getenv("PRICE_DATA")
os.path.join(price_data_path, "**/part.0.parquet")
parquet_files = glob(os.path.join(price_data_path, "**/part.0.parquet"), recursive = True)


In [15]:
dd.read_parquet(parquet_files)

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
npartitions=11207,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,string,string,int32
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [16]:
ddf = dd.read_parquet(parquet_files)


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [18]:


# Add lags for Close and Adj_Close
ddf = ddf.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag=x['Close'].shift(1),
        Adj_Close_lag=x['Adj Close'].shift(1)
    )
)

# Calculate returns based on Adjusted Close
ddf['returns'] = (ddf['Adj Close'] / ddf['Adj_Close_lag']) - 1

# Calculate the day's range (High minus Low)
ddf['hi_lo_range'] = ddf['High'] - ddf['Low']

# Assign the result to dd_feat
dd_feat = ddf


/var/folders/sj/cjqbtrkn32g5n4lym0w2_y8r0000gn/T/ipykernel_4260/3325031163.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ddf = ddf.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [19]:
# Write your code below.
import pandas as pd

df_feat = dd_feat.compute()
df_feat = df_feat.reset_index(level='ticker')  
df_feat['rolling_avg_return'] = df_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())
df_feat.head()

,ticker,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,rolling_avg_return
0,HUM,2002-01-02,11.80,11.83,11.27,11.50,10.268298,572500,Health Care,Managed Health Care,2002,NaN,NaN,NaN,0.559999,NaN
1,HUM,2002-01-03,11.50,11.69,11.38,11.50,10.268298,693600,Health Care,Managed Health Care,2002,11.50,10.268298,0.000000,0.309999,NaN
2,HUM,2002-01-04,11.55,11.60,11.31,11.55,10.312946,651900,Health Care,Managed Health Care,2002,11.50,10.268298,0.004348,0.290000,NaN
3,HUM,2002-01-07,11.54,11.66,11.40,11.48,10.250441,576400,Health Care,Managed Health Care,2002,11.55,10.312946,-0.006061,0.260000,NaN
4,HUM,2002-01-08,11.40,11.56,11.28,11.46,10.232587,406500,Health Care,Managed Health Care,2002,11.48,10.250441,-0.001742,0.280001,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No, it wasn’t strictly necessary to convert the DataFrame to Pandas to calculate the moving average return. Dask supports rolling window operations like .rolling().mean() on grouped data, so this operation can indeed be performed within Dask itself.

+ Would it have been better to do it in Dask? Why?
  
I think so, because Dask has better scalability in term of handling large dataset,
by leveraging distributed computation.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.